Theorems (or conjectures) for the theory of <a class="ProveItLink" href="theory.ipynb">proveit.physics.quantum.QPE</a>
========

In [ ]:
import proveit
# 72 cols ==============================================================
# Prepare this notebook for defining the theorems of a theory:
%theorems_notebook # Keep this at the top following 'import proveit'.
from proveit import a, b, e, k, l, m, n, r, s, t, u, U, eps
from proveit import IndexedVar, ExprRange,  Lambda
from proveit.logic import (Implies, And, Equals, Forall, Exists, InSet, InClass, 
                           NotEquals, NotInSet, SubsetEq, Difference, SetOfAll, Disjoint,
                           Bijections)
from proveit.numbers import (
    zero, one, two, three, four, Interval, IntervalCC, IntervalCO, IntervalOO, IntervalOC,
    Integer, IntegerNonZero, IntegerNeg, Natural, NaturalPos, Complex, Real, RealNonNeg,
    Abs, Add, Neg, subtract, ModAbs, Exp, exp, frac, Mult, Div, sqrt, sqrd, Sum,
    greater, greater_eq, Less, LessEq, Mod, i, pi, exp2pi_i)
from proveit.linear_algebra import (
    MatrixMult, ScalarMult, Unitary, TensorProd, VecAdd, VecSum, Norm)
from proveit.statistics import Prob, ProbOfAll, SampleSpaces
from proveit.physics.quantum import Bra, Meas, NumBra, NumKet, Qmult, QubitRegisterSpace
from proveit.physics.quantum import ket0, ket1, var_ket_u, inv_root2, normalized_var_ket_u
from proveit.physics.quantum.QPE import (
    _U, _ket_u, _n, _t, phase, _phase, _phase_est, phase_est, _phase_in_interval,
    _eps, _s, _b, _delta,    
    two_pow_t, _two_pow_t, _alpha, _alpha_l, _alpha_l_sqrd,
    _s_in_nat_pos, _t_in_natural_pos, _ideal_phase_cond,
    _diff_l_scaled_delta, _full_domain, _full_domain_sans_zero, _neg_domain,
    _pos_domain, _e_domain, _U_pow_two_pow_k,
    _Psi_ket, _psi, _psi_t_ket, s_ket_domain, two_pow_s,
    _psi_t_circuit, _Psi_circuit, _phase_circuit, _phase_est_circuit, _n_ge_two, _t_req,
    m_domain, _Omega, _sample_space, _success_cond)
#    _success_prob_guarantee, _t_req)
from proveit.physics.quantum.QPE.phase_est_ops import (
    Psuccess, Pfail, ModAdd, SubIndexed, exp2pi_i_on_two_pow_t, exp_neg_2pi_i_on_two_pow_t)
from proveit.physics.quantum.QFT import InverseFourierTransform
from proveit.trigonometry import Sin
from IPython.display import display

In [ ]:
%begin theorems

## The main theorems: QPE circuit output

These main theorems are proven using the "local theorems" below via "literal generalization" in which we convert literals to variables and eliminate extraneous theorem/axiom dependencies.  That is, local definitions (e.g., definitions for $t$, $U$, $\oplus$, $P_{\textrm fail}$, etc. in the axioms notebook) are used for convenience to prove local theorems but are formally eliminated to prove these statements that quantify over $t$, $U$, etc. and become logically independent of the local definitions.

In [ ]:
qpe_exact = Forall(
    (s, t), Forall(
        U, Forall(
            var_ket_u, Forall(
                phase,
                Equals(Prob(_phase_circuit.literals_as_variables(_s, _t, _U, _ket_u, _phase)), one),
                domain=Real,
                conditions=[_ideal_phase_cond.literals_as_variables(_t, _phase), 
                            Equals(MatrixMult(U, var_ket_u),
                                   ScalarMult(exp2pi_i(phase), var_ket_u))]).with_wrapping(),
            domain=s_ket_domain, condition=normalized_var_ket_u),
        domain=Unitary(two_pow_s)),
    domain=NaturalPos)

In [ ]:
qpe_guarantee = Forall(
    eps, Forall(
        (s, n), Forall(
            (U, var_ket_u, phase), 
            Exists(
                t, greater_eq(
                    ProbOfAll(m, _phase_est_circuit.literals_as_variables(_s, _t, _U, _ket_u, _phase),
                              domain=m_domain.literals_as_variables(_t), 
                              condition=_success_cond.literals_as_variables(_t, _n, _phase))
                    .with_wrapping(), 
                    subtract(one, _eps.as_variable())),
                domain=NaturalPos,
                condition=_t_req.literals_as_variables(_t, _n, _eps)).with_wrapping(),
            domains=(Unitary(two_pow_s), s_ket_domain, Real),
            conditions=[_phase_in_interval.literals_as_variables(_phase),
                        normalized_var_ket_u,
                        Equals(MatrixMult(U, var_ket_u),
                               ScalarMult(exp2pi_i(phase), var_ket_u))]).with_wrapping(),
        domain=NaturalPos, condition=_n_ge_two.literals_as_variables(_n)),
    domain=IntervalOC(zero, one))

## Local theorems (for convenience - used only internally)

### Let's match quantum circuit outputs with definitions for $|\psi_t \rangle$ and $|\Psi\rangle$.

In [ ]:
_psi_t_output = Forall(t, Equals(Prob(_psi_t_circuit.literals_as_variables(_t)), one), 
                       domain=NaturalPos)

In [ ]:
_Psi_output = Equals(Prob(_Psi_circuit), one)

### Establish the probabilistic sample space of measurement outcomes.

In [ ]:
_sample_space_bijection = (
    InSet(Lambda(_sample_space.instance_param, _sample_space.instance_expr), 
          Bijections(_sample_space.domain, _Omega)))

In [ ]:
_outcome_prob = Forall(
    m, Equals(Prob(_phase_est_circuit), sqrd(Abs(Qmult(NumBra(m, _t), _Psi_ket)))),
    domain=m_domain)

In [ ]:
_Omega_is_sample_space = InClass(_Omega, SampleSpaces)

In [ ]:
_outcome_prob_vs_best = Forall(
    l, Equals(Prob(_phase_est_circuit.basic_replaced({m:ModAdd(_b, l)})), _alpha_l_sqrd),
           domain=_full_domain)

### Simplify  $|\psi_t \rangle$ and evaluate $\alpha_l \equiv \lvert b \oplus l \rangle$ as the amplitudes of $\lvert \Psi \rangle$ relative to the $t$-bit estimate of $2^t \varphi$, $b$.

This theorem captures Nielsen & Chuang's formula 5.20 (pg 222).

In [ ]:
# Notice we're using a ScalarMults instead of QMults on the rhs
# at least until QMults are automatically simplified to do that instead
_psi_t_var_formula = Forall(
    t, Equals(_psi_t_ket,
              ScalarMult(frac(one, Exp(two, frac(t, two))),
                    VecSum(k, ScalarMult(exp(Mult(two, pi, i, _phase, k)), NumKet(k, t)),
                           domain=Interval(zero, subtract(Exp(two, t), one))))),
    domain=NaturalPos)

This theorem is related to Nielsen & Chuang's formula 5.23 and 5.24 (pg 223).  It is the implicit step in between these.

In [ ]:
_alpha_l_eval = Forall(
        l,
        Equals(_alpha_l,
               Mult(frac(one, _two_pow_t),
                   Sum(k, Mult(exp_neg_2pi_i_on_two_pow_t(k, ModAdd(_b, l)),
                               exp2pi_i(_phase, k)),
                       domain=Interval(zero, subtract(_two_pow_t, one))))),
        domain=Integer)

In [ ]:
# Depends upon the _ideal_phase_cond axiom that is only applicable in the ideal case
# when the phase is exactly represented in t bits.
_ideal_alpha_0 = Equals(SubIndexed(_alpha, zero), one)

#### *Evaluation of $\alpha_l$ after performing the geometric series summation in terms of $\delta$:*

This theorem is Nielsen & Chuang's formula 5.26 (pg 224).

In [ ]:
_alpha_l_summed = Forall(
    l,
    Equals(_alpha_l,
           Mult(frac(one, _two_pow_t),
                    frac(subtract(one, exp2pi_i(subtract(Mult(_two_pow_t, _delta), l))),
                         subtract(one, exp2pi_i(subtract(_delta, frac(l, _two_pow_t))))))),
    domain=_full_domain,
    conditions = [NotEquals(l, zero)])

### Compute the probability of failure with a measurement beyond $e$ units away from $b$.

This theorem is represents Nielsen & Chuang's formula 5.27 (pg 224).

In [ ]:
_fail_sum = (
    Forall(
        e,
        Equals(Pfail(e),#LessEq(Pfail(e),
               Add(Sum(l, _alpha_l_sqrd, domain=_neg_domain),
                   Sum(l, _alpha_l_sqrd, domain=_pos_domain))),
        domain=_e_domain))

This theorem is related to Nielsen & Chuang's formula 5.29 (pg 224).

In [ ]:
_alpha_l_sqrd_ineq = Forall(
    l,
    LessEq(_alpha_l_sqrd,
           frac(one,
                Mult(four, Exp(_diff_l_scaled_delta, two)))),
    domain=_full_domain_sans_zero)

#### Obtain a bound on the failure probability:

This theorem represents Nielsen & Chuang's formula 5.30 (pg 224).

In [ ]:
_fail_ineq_lemma = Forall(
    e,
    LessEq(Pfail(e), 
           Mult(frac(one, four), 
                Add(Sum(l, frac(one, sqrd(_diff_l_scaled_delta)), domain=_neg_domain),
                    Sum(l, frac(one, sqrd(_diff_l_scaled_delta)), domain=_pos_domain)))), 
    domain=_e_domain)

This theorem represents Nielsen & Chuang's formula 5.34 (pg 224).

In [ ]:
_fail_ineq = Forall(
    e,
    LessEq(Pfail(e), frac(one, Mult(two, subtract(e, one)))),
    domain=_e_domain)

### Evaluate the success probability as the complement of failure and establish our success guarantee.

In [ ]:
_success_complements_failure  = Forall(
        e, Equals(Psuccess(e), subtract(one, Pfail(e))),
        domain=_e_domain)

Taking Nielsen & Chuang's formula 5.35 (pg 224) as an axiom for the number of qubits in the top register, $t = \left(n + \left\lceil \textrm{log}_2\left(2 + \frac{1}{2 \cdot \epsilon}\right)\right\rceil\right)$, we derive a success probability greater than $1 - \epsilon$.  That is, rather then deriving the number of qubits needed for a certain precision and success probability as done by Nielsen & Chuang, we derive that a given number of qubits is sufficient.

In [ ]:
_success_prob_guarantee = greater_eq(
    ProbOfAll(m, _phase_est_circuit,
              domain=m_domain, condition=_success_cond).with_wrapping(), 
    subtract(one, _eps))

### Minor theorems used through-out

In [ ]:
_psi_t_ket_is_normalized_vec = Forall(
    t, And(InSet(_psi_t_ket, QubitRegisterSpace(t)),
           Equals(Norm(_psi_t_ket), one)),
    domain=NaturalPos)

In [ ]:
_Psi_ket_is_normalized_vec = And(InSet(_Psi_ket, QubitRegisterSpace(_t)),
                                  Equals(Norm(_Psi_ket), one))

#### Take care of number domain issues:

In [ ]:
_best_is_int = InSet(_b, Integer)

In [ ]:
# t (represented by the Literal _t) denotes
# the number of Qbits in the input register
# Thus, 2^t is a positive natural number
_two_pow_t_is_nat_pos = InSet(_two_pow_t, NaturalPos)

In [ ]:
# t (represented by the Literal _t) denotes
# the number of Qbits in the input register
_two_pow_t_minus_one_is_nat_pos = InSet(Exp(two, subtract(_t, one)), NaturalPos)

In [ ]:
# t (represented by the Literal _t) denotes
# the number of Qbits in the input register
_two_pow_t_less_one_is_nat_pos = InSet(subtract(_two_pow_t, one), NaturalPos)

In [ ]:
# t (represented by the Literal _t) denotes
# the number of Qbits in the input register
_two_pow_t_not_zero = NotEquals(_two_pow_t, zero)

In [ ]:
# n (represented by the Literal _n) denotes the desired number
# of bits of precision, and thus n ≤ t)
_n_less_eq_t = LessEq(_n, _t)

In [ ]:
# t (represented by the Literal _t) denotes the number of Qbits in the input register
# n (represented by the Literal _n) denotes the desired number of bits of precision
# In Nielsen & Chuang, e = 2^{t-n} - 1 is the max desired error
_e_value_ge_two = greater_eq(subtract(Exp(two, subtract(_t, _n)), one), two)

In [ ]:
_e_value_in_e_domain = InSet(subtract(Exp(two, subtract(_t, _n)), one), _e_domain)

In [ ]:
# The o-plus addition denotes addition modulo 2^t, resulting in an integer
_mod_add_closure = Forall((a, b), InSet(ModAdd(a, b), 
                                        Interval(zero, subtract(Exp(two, _t), one))), 
                         domain=Integer)

In [ ]:
# The phase phi is in the real interval [0, 1)
_phase_is_real = InSet(_phase, Real)

In [ ]:
_pos_domain_in_full_domain = Forall(
        e, Forall(l, InSet(l, _full_domain), domain=_pos_domain),
        domain=NaturalPos)

In [ ]:
_pos_domain_in_full_domain_sans_zero = Forall(
        e, SubsetEq(_pos_domain, _full_domain_sans_zero),
        domain=_e_domain)

In [ ]:
_neg_domain_in_full_domain = Forall(
        e, Forall(l, InSet(l, _full_domain), domain=_neg_domain),
        domain=NaturalPos)

In [ ]:
_neg_domain_in_full_domain_sans_zero = Forall(
        e, SubsetEq(_neg_domain, _full_domain_sans_zero),
        domain=_e_domain)

In [ ]:
_pos_domain_within_integer = Forall(
        e, SubsetEq(_pos_domain, Integer),
        domain=_e_domain)

In [ ]:
_neg_domain_within_integer = Forall(
        e, SubsetEq(_neg_domain, Integer),
        domain=_e_domain)

In [ ]:
_delta_is_real = InSet(_delta, Real)

#### This derives from $\delta$ being the difference between $\delta$ and its best $t$-bit estimate (without going over):

In [ ]:
_scaled_delta_in_interval = InSet(Mult(_two_pow_t, _delta), IntervalCO(zero, one))

In [ ]:
_all_alpha_l_is_complex = Forall(l, InSet(_alpha_l, Complex), domain=Integer)

#### Follows from scaled_delta_in_interval:

In [ ]:
_scaled_delta_not_eq_nonzeroInt = Forall(
        l, NotEquals(Mult(_two_pow_t, _delta), l),
        domain=Integer, conditions = [NotEquals(l, zero)])

In [ ]:
_delta_not_eq_scaledNonzeroInt = Forall(
        l, NotEquals(_delta, frac(l, _two_pow_t)),
        domain=Integer, conditions = [NotEquals(l, zero)])

In [ ]:
_delta_diff_in_interval = Forall(
        l,
        InSet(subtract(_delta, frac(l, _two_pow_t)),
              IntervalCO(Neg(frac(one, two)), frac(one, two))),
        domain=_full_domain)

In [ ]:
_scaled_delta_diff_in_interval = Forall(
        l,
        InSet(Mult(two, pi, subtract(_delta, frac(l, _two_pow_t))),
              IntervalCC(Neg(pi), pi)),
        domain=_full_domain)

In [ ]:
_non_int_delta_diff = Forall(
        l,
        NotInSet(subtract(_delta, frac(l, _two_pow_t)),
                Integer), 
        domain=_full_domain,
        conditions = [NotEquals(l, zero)])

In [ ]:
# use alternative below instead of this one?
# Notice the condition in the alternate that l ≠ 0
_scaled_abs_delta_diff_interval = Forall(
        l,
        InSet(Mult(pi, Abs(subtract(_delta, frac(l, _two_pow_t)))),
              IntervalOC(zero, Div(pi, two))),
        domain=_full_domain,
        conditions = [NotEquals(l, zero)])

In [ ]:
_pfail_in_real = Forall(e, InSet(Pfail(e), Real), domain=_e_domain)

#### *Modulo addition may be converted to regular addition within $2 \pi i$ exponentiation:*

In [ ]:
_exp2pi_i_modadd = Forall(
        (a, b),
        Equals(exp2pi_i_on_two_pow_t(ModAdd(a, b)), 
               exp2pi_i_on_two_pow_t(Add(a, b))),
        domain=Integer)

In [ ]:
_phase_from_best = Equals(_phase, Add(frac(_b, _two_pow_t), _delta))

In [ ]:
_scaled_delta_minus_l__in__real = Forall(l, InSet(subtract(Mult(_two_pow_t, _delta), l), Real),
                                         domain = Integer)

In [ ]:
_delta_diff_exp_not_one = Forall(
    l,
    NotEquals(exp2pi_i(subtract(_delta, frac(l, _two_pow_t))),
          one),
    domain=_full_domain,
    conditions = [NotEquals(l, zero)])

In [ ]:
_modabs_in_full_domain_simp = Forall(l, Equals(ModAbs(l, _two_pow_t), Abs(l)),
                                     domain=_full_domain)

In [ ]:
_pos_domain_within_natpos = Forall(e, SubsetEq(_pos_domain, NaturalPos), domain=_e_domain)

In [ ]:
_neg_domain_within_negint = Forall(e, SubsetEq(_neg_domain, IntegerNeg), domain=_e_domain)

In [ ]:
%end theorems